In [1]:
import os
import shutil
os.environ['CUDA_VISIBLE_DEVICES'] = '3'

import tensorflow as tf
from tensorflow.keras import backend as keras_backend
import numpy as np

gpu_on = True

if gpu_on :
    gpu_devices = tf.config.experimental.list_physical_devices("GPU")
    for device in gpu_devices:
        tf.config.experimental.set_memory_growth(device, True)
else:
    os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
    gpu_devices = tf.config.experimental.list_physical_devices("CPU")

print(gpu_devices)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
from src.data_loader.classification import ClassifyDataloader
from tensorflow.keras.applications.inception_v3 import preprocess_input
from glob import glob

task = "classification"
data_set_name = "detect_lvi"
batch_size = 16
on_memory = False
argumentation_proba = 0.8
# target_size = (512,512)
target_size = None
interpolation = "bilinear"
class_mode = "binary"
# class_mode = "categorical"
dtype="float32"

train_image_path_list = glob(f"./datasets/{task}/{data_set_name}/train/*/*")
valid_image_path_list = glob(f"./datasets/{task}/{data_set_name}/valid/*/*")
test_image_path_list = glob(f"./datasets/{task}/{data_set_name}/test/*/*")
label_list = os.listdir(f"./datasets/{task}/{data_set_name}/train")

label_to_index_dict = {label:index for index, label in enumerate(label_list)}
index_to_label_dict = {index:label for index, label in enumerate(label_list)}

train_data_loader = ClassifyDataloader(image_path_list=train_image_path_list,
                                       label_to_index_dict=label_to_index_dict,
                                       batch_size=batch_size,
                                       on_memory=on_memory,
                                       argumentation_proba=argumentation_proba,
                                       preprocess_input=preprocess_input,
                                       target_size=target_size,
                                       interpolation=interpolation,
                                       shuffle=True,
                                       class_mode=class_mode,
                                       dtype=dtype
)
valid_data_loader = ClassifyDataloader(image_path_list=valid_image_path_list,
                                       label_to_index_dict=label_to_index_dict,
                                       batch_size=batch_size,
                                       on_memory=on_memory,
                                       argumentation_proba=0,
                                       preprocess_input=preprocess_input,
                                       target_size=target_size,
                                       interpolation=interpolation,                                       
                                       shuffle=False,
                                       class_mode=class_mode,
                                       dtype=dtype
)
test_data_loader = ClassifyDataloader(image_path_list=test_image_path_list,
                                       label_to_index_dict=label_to_index_dict,
                                       batch_size=1,
                                       on_memory=False,
                                       argumentation_proba=0,
                                       preprocess_input=preprocess_input,
                                       target_size=target_size,
                                       interpolation=interpolation,                                       
                                       shuffle=False,
                                       class_mode=class_mode,
                                       dtype=dtype
)

Total data num 8608 with 2 classes
Total data num 1609 with 2 classes
Total data num 1469 with 2 classes


In [3]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.losses import CategoricalCrossentropy, BinaryCrossentropy
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model

import numpy as np
np.random.seed(1337)  # for reproducibility

DROPOUT_RATIO = 0.5

grad_cam = True
transfer_learning = False
epoch_release_frozen = 10
transfer_train_mode = "include_deep_layer"
layer_name_frozen_to = "mixed4"

#  binary_sigmoid, categorical_sigmoid, categorical_softmax
activation = "binary_sigmoid"

# create the base pre-trained model~
base_model = InceptionV3(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    classes=None,
    pooling=None,
    classifier_activation=None
)

if transfer_learning:
    if transfer_train_mode == "dense_only":
        base_model.trainable = False
    elif transfer_train_mode == "include_deep_layer":
        for layer in base_model.layers: 
            layer.trainable = False
            if layer.name == layer_name_frozen_to:
                break

# add a global spatial average pooling layer
x = base_model.output
# (Batch_Size,?)
x = GlobalAveragePooling2D()(x)
x = Dropout(DROPOUT_RATIO)(x)
# let's add a fully-connected layer
# (Batch_Size,1)
x = Dense(1024, activation='relu')(x)
# (Batch_Size,1024)
x = Dropout(DROPOUT_RATIO)(x)

if grad_cam:
    x *= 1e-1
    keras_backend.set_floatx('float64')
    dense_dtype = "float64"
else:
    dense_dtype = "float32"
    
if activation == "binary_sigmoid":
    predictions = Dense(1, activation='sigmoid', dtype=dense_dtype)(x)
    loss_function = BinaryCrossentropy(label_smoothing=0.01)
elif activation == "categorical_sigmoid":
    predictions = Dense(2, activation='sigmoid', dtype=dense_dtype)(x)
    loss_function = CategoricalCrossentropy(label_smoothing=0.01)
elif activation == "categorical_softmax":
    predictions = Dense(2, activation='softmax', dtype=dense_dtype)(x)
    loss_function = CategoricalCrossentropy(label_smoothing=0.01)

# this is the model we will train
model = Model(base_model.input, predictions)

# imported_weight_path = "./weights/classification/LVI_detect_classfication/binary_sigmoid/weights_1.0_1.0_InceptionV3_Imagenet_LVI_level2/weights_129_0.0012.hdf5"
# model.load_weights(imported_weight_path)

87916544/87910968 [==============================] - 2s 0us/step


In [4]:
from datetime import date

from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.optimizers import Nadam

today = date.today()

# YY/MM/dd
today_str = today.strftime("%Y-%m-%d")
today_weight_path = f"./weights/{task}/{data_set_name}/{today_str}/target_size_{target_size}/" 
today_logs_path = f"./logs/{task}/{data_set_name}/{today_str}/target_size_{target_size}/"
os.makedirs(today_weight_path, exist_ok=True)
os.makedirs(today_logs_path, exist_ok=True)
optimizer = Nadam(1e-3, clipnorm=1)

save_c = ModelCheckpoint(
    today_weight_path+"/weights_{val_loss:.4f}_{loss:.4f}_{epoch:02d}.hdf5",
    monitor='val_loss',
    verbose=0,
    save_best_only=False,
    save_weights_only=True,
    mode='min')


reduceLROnPlat = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.1,
    patience=20,
    verbose=1,
    mode="auto",
    min_delta=0.0001,
    cooldown=5,
    min_lr=1e-7)
csv_logger = CSVLogger(f'{today_logs_path}/log.csv', append=False, separator=',')


def make_model_trainable(epoch, model, target_epoch):
    if epoch > target_epoch:
        for layer in model.layers:
            layer.trainable = True
            
make_trainable_callback = LambdaCallback(
    on_epoch_begin=lambda epoch,logs: make_model_trainable(epoch, model, target_epoch=epoch_release_frozen)
)

model.compile(optimizer=optimizer, loss=loss_function, metrics=['accuracy'])

In [5]:
# start_epoch = 0
# epochs = 200

# model.fit(
#     train_data_loader,
#     validation_data=valid_data_loader,
#     epochs=epochs,
#     callbacks=[reduceLROnPlat, save_c, csv_logger],
#     initial_epoch=start_epoch
# )

# Class Activation Map

In [6]:
import cv2
import os
import numpy as np

def imread(img_path, channel=None):
    img_byte_stream = open(img_path.encode("utf-8"), "rb")
    img_byte_array = bytearray(img_byte_stream.read())
    img_numpy_array = np.asarray(img_byte_array, dtype=np.uint8)
    img_numpy_array = cv2.imdecode(
        img_numpy_array, cv2.IMREAD_UNCHANGED)
    if channel == "rgb":
        img_numpy_array = cv2.cvtColor(
            img_numpy_array, cv2.COLOR_BGR2RGB)
    if len(img_numpy_array.shape) == 2:
        img_numpy_array = np.expand_dims(img_numpy_array, axis=-1)
    return img_numpy_array

from tensorflow import keras
import matplotlib.cm as cm

def get_last_conv_name(model):
    layer_names = [layer.name for layer in model.layers]
    conv_layer_name = [layer_name for layer_name in layer_names if layer_name.find("conv") >= 0]
    last_conv_name = conv_layer_name[-1]
    
    return last_conv_name

def get_img_array(img_path, channel="rgb"):
    
    img_array = imread(img_path, channel="rgb")
    img_array = np.expand_dims(img_array, axis=0)
    
    return img_array

def make_grad_model(model, target_layer_name):
    # Model input: model.input
    # Model output: [last_conv_layer_outputs, model.outputs]
    grad_model = keras.models.Model(
        model.input, [model.get_layer(target_layer_name).output, model.output]
    )
    return grad_model

def make_gradcam_heatmap(img_array, grad_model, pred_index=None):
    # compute
    # 1. last_conv_layer_output
    # 2. class_channel value in prediction
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # compute gradient 
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # compute gradient channel mean 
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    
    # compute matmul(last_conv_layer_output, pooled_grads)
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

def save_and_display_gradcam(img_array_normalized, heatmap, cam_path="cam.jpg", alpha=0.4, display_cam=True):
    # Load the original image
    img_array = (img_array_normalized + 1) * 127.5
    img_array = img_array.astype('uint8')
    img_array = img_array[0]
    # Rescale heatmap to a range 0-255
    heatmap = np.uint8(255 * heatmap)

    # Use jet colormap to colorize heatmap
    jet = cm.get_cmap("jet")

    # Use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # Create an image with RGB colorized heatmap
    jet_heatmap = keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img_array.shape[1], img_array.shape[0]))
    jet_heatmap = keras.preprocessing.image.img_to_array(jet_heatmap)

    # Superimpose the heatmap on original image
    superimposed_img = jet_heatmap * alpha + img_array
    superimposed_img = keras.preprocessing.image.array_to_img(superimposed_img)

    # Save the superimposed image
    superimposed_img.save(cam_path)

    # Display Grad CAM
    if display_cam:
        display(Image.open(cam_path))


def decode_classify_predictions(pred_array, index_to_label_dict):
    predicted_index = np.sum(np.argmax(preds, axis=1))
    predicted = index_to_label_dict[predicted_index]
    
    return predicted

def decode_binary_classify_predictions(pred_array):
    predicted = np.round(pred_array).astype("int32")
    predicted = predicted.flatten()[0]
    return predicted

def restore_sigmoid_value(y, decrease_ratio=1e1):
    original_x = np.log(y/(1-y))
    restored_x = decrease_ratio*original_x
    new_y = 1/(1+np.exp(-restored_x))
    return new_y


In [7]:
def remove_folder_png_files(folder_path):
    regxp_files = os.path.join(folder_path, "*.png")
    target_files = glob(regxp_files)
    for target_file in target_files:
        os.remove(target_file)

In [8]:
layer_names = [layer.name for layer in model.layers]
conv_layer_name = [layer_name for layer_name in layer_names if layer_name.find("conv") >= 0]
last_conv_name = conv_layer_name[-1]

print(last_conv_name)

conv2d_93


In [9]:
weight_path_list = glob(f"./weights/classification/detect_lvi/2021-06-28/target_size_None/*.hdf5")
weight_path_list.sort()
best_path = weight_path_list[0]
print(best_path)

./weights/classification/detect_lvi/2021-06-28/target_size_None/weights_0.2390_0.1356_84.hdf5


In [10]:
target_data_loader = test_data_loader

target_data_loader.batch_size = 1

model_predicted = model.predict(target_data_loader)
ground_truth = np.array([label for img_array, label in target_data_loader])

model.evaluate(target_data_loader)

1469/1469 [==============================] - 108s 72ms/step - loss: 0.6897 - accuracy: 0.5255


[0.6897123047005489, 0.5255275697753574]

In [11]:
negative_group_1 = np.where(model_predicted < 0.5)[0]
negative_group_2 = np.where(model_predicted != 0)[0]
negative_confirm = np.where(model_predicted == 0)[0]
negative_not_confirm = np.intersect1d(negative_group_1, negative_group_2)

positive_group_1 = np.where(model_predicted > 0.5)[0]
positive_group_2 = np.where(model_predicted != 1)[0]
postive_confirm = np.where(model_predicted == 1)[0]
postive_not_confirm = np.intersect1d(positive_group_1, positive_group_2)

ground_truth_negative = np.where(ground_truth == 0)[0]
ground_truth_positive = np.where(ground_truth == 1)[0]

true_negative = np.intersect1d(negative_group_1, ground_truth_negative)
true_positive = np.intersect1d(positive_group_1, ground_truth_positive)
false_negative = np.intersect1d(negative_group_1, ground_truth_positive)
false_positive = np.intersect1d(positive_group_1, ground_truth_negative)


In [12]:
print(len(negative_confirm))
print(len(negative_not_confirm))
print(len(postive_confirm))
print(len(postive_not_confirm))

0
82
0
1387


In [13]:
print(len(negative_confirm))
print(len(negative_not_confirm))
print(len(postive_confirm))
print(len(postive_not_confirm))

0
82
0
1387


In [14]:
print(false_negative)
print(false_positive)

[  15   38   42   45   49   56   60   68   81   83   99  101  126  131
  135  138  176  189  200  217  241  245  250  251  255  279  301  324
  335  338  371  372  382  402  403  405  406  415  421  423  427  463
  471  473  484  488  489  490  494  507  521  522  536  543  545  565
  566  570  583  584  587  592  616  618  659  709  728  732  738  740
  743  744  747  756  761  765  766 1333 1346 1347 1352 1372]
[]


In [ ]:
from matplotlib import pyplot as plt

target_layer_name = "mixed10"
target_cam_folder = "./gradcam/"
true_negative_folder = f"{target_cam_folder}/true_negative/"
true_positive_folder = f"{target_cam_folder}/true_positive/"
false_negative_folder = f"{target_cam_folder}/false_negative/"
false_positive_folder = f"{target_cam_folder}/false_positive/"

os.makedirs(true_negative_folder, exist_ok=True)
os.makedirs(true_positive_folder, exist_ok=True)
os.makedirs(false_negative_folder, exist_ok=True)
os.makedirs(false_positive_folder, exist_ok=True)

remove_folder_png_files(true_negative_folder)
remove_folder_png_files(true_positive_folder)
remove_folder_png_files(false_negative_folder)
remove_folder_png_files(false_positive_folder)

grad_model = make_grad_model(model, target_layer_name)

for index, (img_array_normalized, label) in enumerate(target_data_loader):
    
    # Print what the top predicted class is
    preds = model.predict(img_array_normalized)
    preds_value = preds.flatten()[0]
    preds_value = restore_sigmoid_value(preds_value, decrease_ratio=1e1)
    predicted_label = decode_binary_classify_predictions(preds)
    label = decode_binary_classify_predictions(label)
    pred_index = None
    # pred_index = label_to_index_dict[predicted_label]

    print(preds)
    print(preds_value)
    print(f"Predicted: {predicted_label}")

    # Generate class activation heatmap
    heatmap = make_gradcam_heatmap(img_array_normalized, grad_model, pred_index=pred_index)
    
    if predicted_label == 0 and label == 0:
        cam_path = f"{true_negative_folder}/cam_{index}_{preds_value:.4f}.png"
    if predicted_label == 1 and label == 1:
        cam_path = f"{true_positive_folder}/cam_{index}_{preds_value:.4f}.png"
    if predicted_label == 0 and label == 1:
        cam_path = f"{false_negative_folder}/cam_{index}_{preds_value:.4f}.png"
    if predicted_label == 1 and label == 0:
        cam_path = f"{false_positive_folder}/cam_{index}_{preds_value:.4f}.png"
    
    save_and_display_gradcam(img_array_normalized, heatmap, cam_path=cam_path, display_cam=False)
    

[[0.49510723]]
0.451226327543285
Predicted: 0
[[0.49595122]]
0.45959962619361516
Predicted: 0
[[0.49531204]]
0.4532559572642267
Predicted: 0
[[0.49565339]]
0.4566419788110031
Predicted: 0
[[0.49454482]]
0.4456614771227666
Predicted: 0
[[0.49394407]]
0.43973220006165714
Predicted: 0
[[0.49545088]]
0.45463264784239277
Predicted: 0
[[0.49379639]]
0.43827719217272
Predicted: 0
[[0.49289864]]
0.42945536750269536
Predicted: 0
[[0.49528335]]
0.4529714885947403
Predicted: 0
[[0.49510279]]
0.45118239236274404
Predicted: 0
[[0.49329197]]
0.43331537498730854
Predicted: 0
[[0.49368114]]
0.43714235094257636
Predicted: 0
[[0.49450746]]
0.44529229399273024
Predicted: 0
[[0.49358438]]
0.43619007738212134
Predicted: 0
[[0.49504054]]
0.4505657510995602
Predicted: 0
[[0.49426677]]
0.44291513517335057
Predicted: 0
[[0.49437252]]
0.4439592312833213
Predicted: 0
[[0.49327817]]
0.43317978550107195
Predicted: 0
[[0.4963423]]
0.4634874961026869
Predicted: 0
[[0.49526757]]
0.45281506956626133
Predicted: 0
[[0.4

[[0.49709967]]
0.47102886310774017
Predicted: 0
[[0.49255892]]
0.42612836357298567
Predicted: 0
[[0.49505975]]
0.45075607641908005
Predicted: 0
[[0.49534786]]
0.4536110990182939
Predicted: 0
[[0.49173184]]
0.41805651904612146
Predicted: 0
[[0.49291396]]
0.4296055626308815
Predicted: 0
[[0.49449484]]
0.44516756300072524
Predicted: 0
[[0.48967772]]
0.3982050506535208
Predicted: 0
[[0.49603458]]
0.4604279078459056
Predicted: 0
[[0.49578928]]
0.4579910308110509
Predicted: 0
[[0.4936886]]
0.4372157996213106
Predicted: 0
[[0.49394435]]
0.43973496258632294
Predicted: 0
[[0.49120266]]
0.4129144602409672
Predicted: 0
[[0.49527882]]
0.4529265924245531
Predicted: 0
[[0.49108309]]
0.41175520351124045
Predicted: 0
[[0.49348608]]
0.4352232200315719
Predicted: 0
[[0.49697796]]
0.46981596903424083
Predicted: 0
[[0.49463216]]
0.44652487014214665
Predicted: 0
[[0.4922343]]
0.4229553969772566
Predicted: 0
[[0.49405028]]
0.44077924176008465
Predicted: 0
[[0.49185472]]
0.41925311433574836
Predicted: 0
[[0.

[[0.4969929]]
0.46996482801918155
Predicted: 0
[[0.49587999]]
0.45889193475824125
Predicted: 0
[[0.49473787]]
0.4475701848242462
Predicted: 0
[[0.49505952]]
0.4507537251135372
Predicted: 0
[[0.49420482]]
0.44230372913286203
Predicted: 0
[[0.49507346]]
0.4508918762962268
Predicted: 0
[[0.49419712]]
0.4422278165207426
Predicted: 0
[[0.49422794]]
0.44253190293116684
Predicted: 0
[[0.49450016]]
0.4452201591191675
Predicted: 0
[[0.49674965]]
0.467541756260489
Predicted: 0
[[0.49558332]]
0.45594654581319743
Predicted: 0
[[0.4949005]]
0.4491793176292897
Predicted: 0
[[0.49540684]]
0.45419587061116345
Predicted: 0
[[0.49395297]]
0.4398199491847355
Predicted: 0
[[0.49534638]]
0.4535964201296573
Predicted: 0
[[0.49434761]]
0.44371330945683657
Predicted: 0
[[0.49360692]]
0.4364119076488067
Predicted: 0
[[0.49432549]]
0.4434948561223008
Predicted: 0
[[0.49419675]]
0.44222409254600015
Predicted: 0
[[0.49493068]]
0.44947807582950716
Predicted: 0
[[0.49600698]]
0.46015365453974527
Predicted: 0
[[0.49

[[0.49210768]]
0.42171938550034566
Predicted: 0
[[0.49167948]]
0.4175469791665257
Predicted: 0
[[0.49191976]]
0.419886922751673
Predicted: 0
[[0.49566752]]
0.4567822425803056
Predicted: 0
[[0.49390973]]
0.43939372810045174
Predicted: 0
[[0.49394299]]
0.4397215425945202
Predicted: 0
[[0.49648068]]
0.46486421304072
Predicted: 0
[[0.49315295]]
0.43195012738410055
Predicted: 0
[[0.49278344]]
0.4283264107691153
Predicted: 0
[[0.49079128]]
0.408930010079627
Predicted: 0
[[0.49131297]]
0.413984876776382
Predicted: 0
[[0.49337202]]
0.4341019021997882
Predicted: 0
[[0.49045945]]
0.40572460130857857
Predicted: 0
[[0.49295294]]
0.42998778026741663
Predicted: 0
[[0.49552948]]
0.45541240465939414
Predicted: 0
[[0.49247446]]
0.42530216765794904
Predicted: 0
[[0.49347233]]
0.4350880193107326
Predicted: 0
[[0.4929482]]
0.42994130688450766
Predicted: 0
[[0.49369643]]
0.43729288850392195
Predicted: 0
[[0.49561967]]
0.4563073130926583
Predicted: 0
[[0.4904283]]
0.40542411473973705
Predicted: 0
